In [1]:
from datetime import datetime
from credit_risk.entity import DataIngestionConfig,DataIngestionArtifact
from credit_risk.entity.config_entity import DataValidationConfig
from credit_risk.components.data_validation import DataValidation
from credit_risk.entity.config_entity import  TrainingPipelineConfig
from credit_risk.exception import CreditRiskException
from credit_risk.components.data_ingestion import DataIngestion
import os
from credit_risk.pipeline.training import TrainingPipeline

training_pipeline_config = TrainingPipelineConfig()
tr = TrainingPipeline(training_pipeline_config=training_pipeline_config)
tr.start() 

24/09/16 12:39:48 WARN Utils: Your hostname, langchain resolves to a loopback address: 127.0.1.1; using 192.168.0.103 instead (on interface wlp3s0)
24/09/16 12:39:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/09/16 12:39:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


+----------+-------------+---------------------+-----------------+-----------------+----------+---------+-------------+-----------+-------------------+-------------------------+--------------------------+
|person_age|person_income|person_home_ownership|person_emp_length|      loan_intent|loan_grade|loan_amnt|loan_int_rate|loan_status|loan_percent_income|cb_person_default_on_file|cb_person_cred_hist_length|
+----------+-------------+---------------------+-----------------+-----------------+----------+---------+-------------+-----------+-------------------+-------------------------+--------------------------+
|        22|        59000|                 RENT|            123.0|         PERSONAL|         D|    35000|        16.02|          1|               0.59|                        Y|                        3
|        21|         9600|                  OWN|              5.0|        EDUCATION|         B|     1000|        11.14|          0|                0.1|                        N|     